In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import requests
import json
import time
from tqdm.auto import tqdm
import math
import os, os.path
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
import warnings 
from geopy.geocoders import Nominatim
import ast
import re
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import gensim
import gensim.downloader

import matplotlib.pyplot as plt
from gensim.models.phrases import Phrases, Phraser
from gensim.models import KeyedVectors
import seaborn as sns

from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

import os
import urllib.request
from scipy import spatial
from sklearn.manifold import TSNE
from numpy import dot
from numpy.linalg import norm



In [10]:
df = pd.read_csv("geo_100000_200000.csv",low_memory=False, lineterminator='\n')

In [29]:
df = pd.read_csv("companies_tf_idf.csv",low_memory=False, lineterminator='\n')

In [23]:
a = np.array([[1,3,2],[5,4,1],[0,9,2]])

In [25]:
np.array(list(a))

array([[1, 3, 2],
       [5, 4, 1],
       [0, 9, 2]])

In [9]:
#df.head(2000).to_csv(f'companies_translated.csv')

In [30]:
df.head()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,name,description,number_of_employees,seo_description,annual_revenue_estimation,...,geo_lat,industries,keywords,revenue_currency,headquarter_location_geo_code_json,eng_industries,eng_keywords,tokenized_indutries,tokenized_keywords,ind_tf_idf
0,0,0,0,0,10293,Renesas Electronics Corporation,Renesas Electronics Corporation is a Japanese ...,10000,"A global leader in microcontrollers, analog, p...",1.000000e+09,...,37.372658,"['Halbleiter', 'marketing', 'information_techn...","['hardware', 'information technology', 'semico...",NaN,NaN,"['semiconductor', 'marketing', 'information_te...","['hardware', 'information technology', 'semico...","['semiconductor', 'marketing', 'information', ...","['hardware', 'information', 'technology', 'sem...","[2.8538821596168944, -0.9744060588297757, -1.3..."
1,1,1,1,1,10294,Klüber Lubrication München SE & Co. KG,Benefit from our comprehensive range of specia...,5000,"Benefíciese de nuestra consultoría experta, de...",1.000000e+07,...,NaN,"['Chemieindustrie', 'Chemikalien', 'Wälzlagerf...","['ceras lubrificantes', 'lubrificantes especia...",NaN,NaN,"['chemical industry', 'chemicals', 'roller bea...","['lubricating waxes', 'special lubricants', 'i...","['chemical', 'industry', 'chemicals', 'roller'...","['lubricating', 'waxes', 'special', 'lubricant...","[-3.6388067619906956, -2.8767534378903856, -1...."
2,2,2,2,2,10295,RHINE BUSINESS CONSULTING,Als unabhängiges Beratungshaus unterstützen wi...,10,Kompetenz in Business Development und Digitali...,2.000000e+06,...,NaN,"['Management Beratung', 'Management Beratung']","['it interim project management', 'big data am...",NaN,NaN,"['management consulting', 'management consulti...","['it interim project management', 'big data am...","['management', 'consulting', 'management', 'co...","['interim', 'project', 'management', 'big', 'd...","[1.1739061164723017, 0.07123588898358767, -0.0..."
3,3,3,3,3,10296,EMIL,Zahle pro gefahrenem Kilometer Spare bis zu 45...,50,"Versicherer nutzen die EMIL Insurance Suite, u...",1.000000e+07,...,NaN,"['Versicherung', 'Insurance company']",NaN,NaN,NaN,"['insurance', 'Insurance company']",NaN,"['insurance', 'insurance', 'company']",NaN,"[-0.6345912135192034, -0.05205569559852487, -0..."
4,4,4,4,4,10297,Carnival Maritime,Carnival Maritime is the Marine Service Unit o...,500,NaN,5.000000e+07,...,53.542124,"['Seeschiffahrt und maritime Wirtschaft', 'ope...","['maritime', 'maritime]']",NaN,NaN,"['Shipping and maritime economy', 'operations'...","['maritime', 'maritime]']","['shipping', 'maritime', 'economy', 'operation...","['maritime', 'maritime']","[-0.9238364692476992, -0.33482340725601734, -0..."


In [12]:
companies_isna_stats = df.isna().sum() / df.shape[0] * 100
companies_isna_stats

Unnamed: 0                              0.000
id                                      0.000
name                                    0.000
description                             2.469
number_of_employees                     0.022
seo_description                        39.185
annual_revenue_estimation               0.027
address                                 0.691
founding_year                           9.566
languages_spoken_at_company            14.858
geo_lon                                15.402
geo_lat                                15.402
industries                              0.138
keywords                                0.548
revenue_currency                      100.000
headquarter_location_geo_code_json    100.000
Latitude                               66.766
Longitude                              66.766
Full_address                           66.766
dtype: float64

In [9]:
path_glv_50 = 'glove.6B/glove.6B.50d.txt'
embed_glv_50 = get_embedding_dict(path_glv_50)
tqdm.pandas()

In [123]:
df["ind_embed"] = df["eng_industries"].progress_apply(lambda row :  get_row_embedding(row, get_description_embedding_max, embed_glv_50, 50 ))  


  0%|          | 0/2846000 [00:00<?, ?it/s]

In [124]:
df["kw_embed"] = df["eng_keywords"].progress_apply(lambda row :  get_row_embedding(row, get_description_embedding_avg, embed_glv_50, 50 ))  


  0%|          | 0/2846000 [00:00<?, ?it/s]

In [126]:
ind_embeddings = df["ind_embed"].values
kw_embeddings = df["kw_embed"].values
embeddings = []
for i in range(len(ind_embeddings)):
    try:
        embedding = ind_embeddings[i] + kw_embeddings[i]
    except:
        print(kw_embeddings[i])
        print(kw_)

TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'

In [125]:
ind_embeddings

NameError: name 'ind_embeddings' is not defined

In [ ]:
df["embeddings"]

In [31]:
industries_emb = df["ind_tf_idf"].values

In [33]:
ast.literal_eval(industries_emb[0])

[2.8538821596168944,
 -0.9744060588297757,
 -1.3750476134193315,
 -0.28443234997851974,
 -2.71291090601036]

In [17]:
industries_emb.shape

(2000,)

In [20]:
df["tokens_industries"].values[0]

"['semiconductor', 'marketing', 'information', 'technology', 'product', 'management', 'legal', 'business', 'development', 'human', 'resources', 'operations', 'education', 'support', 'finance', 'media', 'commmunication', 'semiconductors', 'engineering', 'sales']"

In [34]:
embedd_list = []
for e in tqdm(list(industries_emb)):
    try:
        embedd_list.append(ast.literal_eval(e))
    except:
        embedd_list.append(list(np.zeros(50)))

  0%|          | 0/2000 [00:00<?, ?it/s]

[[2.8538821596168944,
  -0.9744060588297757,
  -1.3750476134193315,
  -0.28443234997851974,
  -2.71291090601036],
 [-3.6388067619906956,
  -2.8767534378903856,
  -1.4525676502019977,
  2.5393632705505556,
  -2.131277248412279],
 [1.1739061164723017,
  0.07123588898358767,
  -0.016792140355412304,
  0.028975838288398954,
  0.9222789501925202],
 [-0.6345912135192034,
  -0.05205569559852487,
  -0.029813214748315817,
  -0.18846933768231833,
  0.4867613037725389],
 [-0.9238364692476992,
  -0.33482340725601734,
  -0.4212273988580122,
  -0.21478318013949246,
  0.11106740989940153],
 [-0.6345912135192034,
  -0.05205569559852487,
  -0.029813214748315817,
  -0.18846933768231833,
  0.4867613037725389],
 [-0.6345912135192034,
  -0.05205569559852487,
  -0.029813214748315817,
  -0.18846933768231833,
  0.4867613037725389],
 [1.3791379202010041,
  0.04938705858520003,
  -0.10405996385657207,
  -0.048972906476449064,
  -0.11832111888136915],
 [5.538074808951722,
  0.213394112157308,
  -0.04295446760530

In [105]:
kmeans = KMeans(n_clusters=20, random_state=0).fit(np.array(embedd_list))

In [106]:
kmeans = KMeans(n_clusters=20, random_state=0, verbose=True).fit(np.array(embedd_list))

Initialization complete
Iteration 0, inertia 5921472.511018308
Iteration 1, inertia 4276336.228772683
Iteration 2, inertia 4102486.2424195176
Iteration 3, inertia 4036086.4487242484
Iteration 4, inertia 4003852.6827981556
Iteration 5, inertia 3989269.82774539
Iteration 6, inertia 3982111.4418778173
Iteration 7, inertia 3978342.1674371553
Iteration 8, inertia 3977438.392885362
Iteration 9, inertia 3971036.2961184187
Iteration 10, inertia 3969546.4415690624
Iteration 11, inertia 3969350.7998410817
Iteration 12, inertia 3969314.1039630687
Converged at iteration 12: center shift 1.34863313984511e-05 within tolerance 1.5604552531249312e-05.
Initialization complete
Iteration 0, inertia 5374100.892183979
Iteration 1, inertia 4046211.5496991547
Iteration 2, inertia 3979798.405200597
Iteration 3, inertia 3948990.217014883
Iteration 4, inertia 3936655.5639018756
Iteration 5, inertia 3932143.998106585
Iteration 6, inertia 3930525.0635232623
Iteration 7, inertia 3928433.004078993
Iteration 8, iner

In [107]:
examples = give_examples_of_clusters(20, kmeans)

In [109]:
for i in examples[1]:
    print(df["tokens_keywords"].values[i])

['law', 'practice', 'legal', 'services']
nan
['political', 'organization']
['electrical', 'electronic', 'manufacturing']
['professional', 'training', 'coaching']
nan
['facilities', 'services']
['hematology', 'cytology', 'client', 'service', 'pathology', 'phlebotomy', 'clinical', 'chemistry', 'laboratory', 'medicine', 'patient', 'care', 'hospital', 'health', 'care']
nan
['finance']
nan
nan
['hospital', 'health', 'care']
['trainingen', 'sales', 'social', 'media', 'business', 'development', 'communication', 'personal', 'development', 'professional', 'training', 'coaching', 'consumer', 'internet', 'consumers', 'internet', 'information', 'technology', 'services', 'communications']
nan
['live', 'events', 'strategic', 'marketing', 'plan', 'trade', 'shows', 'websites', 'referral', 'marketing', 'brand', 'development', 'advertising', 'multichannel', 'direct', 'marketing', 'national', 'sales', 'meetings', 'ambassador', 'marketing', 'social', 'media', 'roi', 'modeling', 'sales', 'collateral', 'mar

In [20]:
embedd_list

[['[',
  ' ',
  '2',
  '.',
  '8',
  '4',
  '9',
  '1',
  '5',
  '4',
  '0',
  '6',
  ' ',
  '-',
  '0',
  '.',
  '9',
  '7',
  '1',
  '9',
  '0',
  '3',
  '3',
  '6',
  ' ',
  '-',
  '1',
  '.',
  '3',
  '4',
  '9',
  '9',
  '8',
  '9',
  '4',
  '7',
  ' ',
  '-',
  '0',
  '.',
  '2',
  '8',
  '1',
  '1',
  '2',
  '2',
  '9',
  '7',
  ' ',
  '-',
  '2',
  '.',
  '7',
  '5',
  '1',
  '6',
  '7',
  '3',
  '4',
  '4',
  ']'],
 ['[',
  '-',
  '3',
  '.',
  '6',
  '5',
  '5',
  '0',
  '6',
  '9',
  '8',
  '4',
  ' ',
  '-',
  '2',
  '.',
  '8',
  '5',
  '7',
  '4',
  '0',
  '6',
  '2',
  '9',
  ' ',
  '-',
  '1',
  '.',
  '4',
  '0',
  '6',
  '8',
  '2',
  '2',
  '6',
  '1',
  ' ',
  ' ',
  '2',
  '.',
  '4',
  '6',
  '9',
  '2',
  '0',
  '9',
  '5',
  '8',
  ' ',
  '-',
  '2',
  '.',
  '0',
  '4',
  '4',
  '6',
  '1',
  '7',
  '3',
  '6',
  ']'],
 ['[',
  ' ',
  '1',
  '.',
  '1',
  '8',
  '3',
  '5',
  '5',
  '8',
  '3',
  '1',
  ' ',
  ' ',
  '0',
  '.',
  '0',
  '6',
  '3',
  '7',
  '2

In [36]:
neigh = NearestNeighbors(n_neighbors=2, radius=0.4)
#neigh.fit(X)
neigh.fit(np.array(embedd_list))

NearestNeighbors(n_neighbors=2, radius=0.4)

In [42]:
indx = 1
neighbours = neigh.kneighbors([np.array(embedd_list)[indx]], 10, return_distance=False)
print(df["tokenized_indutries"].values[indx])


['chemical', 'industry', 'chemicals', 'roller', 'bearing', 'greases', 'high', 'performance', 'lubricants', 'chain', 'lubricants', 'roller', 'bearing', 'greases', 'specialty', 'lubricants', 'adhesive', 'lubricants', 'lubricants', 'food', 'industry', 'adhesive', 'lubricants', 'chemicals']


In [43]:
for i in neighbours:
    print(df["tokenized_indutries"].values[i])

["['chemical', 'industry', 'chemicals', 'roller', 'bearing', 'greases', 'high', 'performance', 'lubricants', 'chain', 'lubricants', 'roller', 'bearing', 'greases', 'specialty', 'lubricants', 'adhesive', 'lubricants', 'lubricants', 'food', 'industry', 'adhesive', 'lubricants', 'chemicals']"
 "['building', 'construction', 'engineering', 'finance', 'accounting', 'construction']"
 "['consumer', 'goods', 'utilities', 'corrosion', 'protection', 'analytics', 'chemical', 'pipe', 'tapping', 'shut', 'service', 'advice', 'welding', 'technology', 'energy', 'data', 'management', 'edm', 'gas', 'pressure', 'reducing', 'measuring', 'systems', 'combined', 'gas', 'compressor', 'gas', 'meters', 'gas', 'transportation', 'oil', 'energy', 'business', 'development', 'energy', 'supplier', 'oil', 'energy']"
 "['building', 'materials', 'marketing', 'media', 'commmunication', 'renewable', 'energy', 'environment', 'engineering', 'product', 'management', 'support', 'business', 'development', 'operations', 'account

In [110]:
Xtsne = TSNE(n_components=2, verbose=True).fit_transform(np.array(embedd_list))
dftsne = pd.DataFrame(Xtsne)

/Users/ahmedewva/opt/anaconda3/envs/idp/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/ahmedewva/opt/anaconda3/envs/idp/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 2846000 samples in 0.116s...


KeyboardInterrupt: 

In [ ]:
dftsne['cluster'] = kmeans.labels_
dftsne.columns = ['x1','x2','cluster']
fig, ax = plt.subplots(1, figsize=(16,8))
sns.scatterplot(data=dftsne,x='x1',y='x2',hue='cluster',legend="full",  palette="deep",alpha=0.5,ax=ax)
ax.set_title('Visualized on TSNE 2D')

In [114]:
df["ind_embed"].values[0]

1.15910005569458

In [50]:
companies_df = pd.read_csv("companies_with_size.csv",low_memory=False, lineterminator='\n')

In [54]:
 
ids = companies_df["id"].values
embeddings = companies_df["embeddings"].values
sizes = companies_df["sizes"].values

In [52]:
embeddings[0]

'[0.505252221883792, 0.25616556654373807, 0.9160524855057398, 1.171695529586739, 0.08166344163732396, 0.3136875091327561, 0.1517192307316388, -0.15059494599699974, 0.5572942350473669, 0.8443205664451752, 0.956319157862001, 0.45191206131130457, 0.17691833815640873, 0.2546330586903625, 0.4901212567411777, 0.8131875163979001, 0.08591140434145927, 1.0345291880269845, 0.7160233367337949, 0.25490276267131173, 1.0274338747064273, 0.1494356170296669, -0.17083055153489113, -0.032253702601236806, 0.01752521739237839, 0.154226943747037, 0.2696008100691769, -0.08417966754900084, 0.4926450363256865, 0.4858233085316088, 2.2719655856490135, 0.2931341542344954, 0.6675782857669724, 0.3417316179062861, 0.41460063142908943, 0.2443677888562282, -0.02335277828387916, 0.7623227400084337, 0.7611655683981048, 0.4770896910663901, 0.2705167710988058, 0.2739902820645107, 0.4331398498680856, 0.8569078079114357, 0.7184924864106708, 0.31698339629090494, 0.29000305384397507, 0.8616241394645638, 0.6342122128440274, 0

In [55]:
embeddings_list = []
for e in embeddings:
    embeddings_list.append(ast.literal_eval(e))


In [56]:
embeddings = np.array(embeddings_list)


In [57]:
embeddings

array([[ 0.50525222,  0.25616557,  0.91605249, ...,  0.86162414,
         0.63421221,  0.72890161],
       [ 0.46427161, -0.00537723,  0.12487048, ...,  0.33686667,
         0.29060343, -0.04863333],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.51232   , -0.11293799,  0.50338999, ...,  1.052068  ,
         0.68753699,  0.78922603],
       [ 0.57955003,  0.117305  ,  0.53834999, ...,  0.77424997,
         0.77219999,  0.68730003],
       [ 0.15825077,  0.04136539,  0.66390768, ...,  0.46252856,
         0.11531692,  0.42294462]])

# Recommender System


This repo is an implementation of the companies recommender system built to be used by the tendex platform. There are mainly nine python files. The output of each python file is a csv file containing the resulting data sets. The files should be executed sequentially with the output of one being the input of another.


&nbsp;
## Requirements
First install the requirements:
``` 
pip install -r requirements.txt
```

&nbsp;
## Contents
- [Fetching data and creating csv file](#fetching-data-and-creating-csv-file)
- [Translation](#translation)
- [Processing text](#processing-text)
- [Embedding](#embedding)
- [Tf-idf](#tf-idf)
- [Size](#size)
- [Geo-data processing](#geo-data-processing)
- [Clustering](#clustering)
- [Recommender](#recommender)

**Note:** All following files include --begining and --ending parameters so that the execution could be done on a smaller portion of the dataset to decrease the processing time.


&nbsp;
## Fetching data and creating csv file

For this the default url is used in the file. In each step 1000 companies are fetched. The output is a csv containing in each row a company and for each column a feature. The selected features are: "id", "name", "description", "number_of_employees", "seo_description", "annual_revenue_estimation", "revenue_currency", "address", "founding_year", "languages_spoken_at_company", "headquarter_location_geo_code_json", "geo_lon", "geo_lat", "industries", "keywords". Features with many missing values could be dropped.
``` 
python fetching_data.py --num_steps=10 
```

&nbsp;
## Translation

This script does the translations to english for the columns: industries and keywords.
``` 
python translation.py --ending=10000  
```

&nbsp;
## Processing text
This script does the tokenization, lemmatization, removal of non alphanumerical characters and stop words.

``` 
python embeddings.py --df_name="companies_translated.csv"  --ending=10000    
```

&nbsp;
## Embedding
This script computes the average word embeddings for the columns: industries and keywords and sum the results to have one embedding vector for each company.
``` 
python embedding.py --df_name="companies_tokenized.csv"  --ending=10000 
```

&nbsp;
## Tf-idf

This script computes a tf-idf vector per company based on the industry tags.
``` 
tf_idf.py --df_name="companies_tokenized.csv"  --ending=10000 
```

&nbsp;
## Size
This script assign one size value for each company based it's number of employees and it's revenues. 0: small, 1: medium and 2: big
``` 
 python size.py --df_name="companies_embeddings.csv" --ending=10000
```

&nbsp;
## Geo-data processing
This script generates the latitude, longitude and full address for company or nan if the address is not found.
``` 
python geo_processing.py --df_name="companies_full.csv" --ending=10000
```

&nbsp;
## Clustering

This script computes the clustering based on embeddings (tfidf could be also specified). There are two clustering methods to choose from. The output is the cluster number for each company
``` 
python clustering.py --df_name="companies_embeddings.csv" --num_clusters=10 --ending=10000
```

&nbsp;
## Recommender
This script generate for each company a list of similar companies. The recommendation is based on the semantic informations (industries and keywords), size and location.
``` 
python recommendations.py --df_name="companies_with_size.csv"  --ending=10000
```